In [4]:
# Code for ingestion of the dataset

import time
import requests
import pandas as pd
#%pip install yfinance
import yfinance as yf
from pathlib import Path


# Locate repo root and Data folder

def find_repo_root(start_path=None):
    """Walk up directories until .git is found"""
    p = Path(start_path or Path.cwd()).resolve()
    for parent in [p] + list(p.parents):
        if (parent / ".git").exists():
            return parent
    # fallback to current working directory
    return p

REPO_ROOT = find_repo_root()
DATA_DIR = REPO_ROOT / "Data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Output file (csv inside a zip)
output_file = "stock_symbols_jenice_testing2.csv.zip"
output_path = DATA_DIR / output_file


# SEC Parameters

SEC_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
SEC_HEADERS = {"User-Agent": "YourAppName your_email@example.com"}  # required by SEC

def get_universe_from_sec(limit):
    r = requests.get(SEC_URL, headers=SEC_HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()
    df = pd.DataFrame(j["data"], columns=j["fields"])
    df = df.rename(columns={"ticker": "symbol", "name": "company_name"})
    df["symbol"] = df["symbol"].str.upper()
    return df[["symbol", "company_name"]].drop_duplicates().head(limit)

def yf_fetch_info(symbol: str) -> dict:
    candidates = [symbol, symbol.replace("-", ".")]
    for sym in candidates:
        try:
            t = yf.Ticker(sym)
            info = t.get_info()
            if info and isinstance(info, dict) and info.get("quoteType") in ("EQUITY", "ETF"):
                return info
        except Exception:
            pass
    return {}

def build_base_table(limit, sleep_s=0.35):
    universe = get_universe_from_sec(limit=limit)
    print("Symbols pulled:", len(universe.index))
    rows = []
    count = 0
    for sym in universe["symbol"].tolist():
        rows.append(yf_fetch_info(sym))
        count += 1
        time.sleep(sleep_s)
        if count % 200 == 0:
            print(f"{count} rows completed")

    facts = pd.DataFrame(rows)

    df_cols = list(facts.columns)
    added = ["symbol", "company_name"]
    cols = df_cols + added

    base = (
        universe
        .merge(facts, on="symbol", how="left")
        [cols]
        .drop_duplicates(subset=["symbol"])
    )
    return base

# -------------------------------
# Run + Save
# -------------------------------
df_base = build_base_table(limit=200, sleep_s=0.35)

print(df_base.head(10))
print(df_base.shape)
print(df_base.columns)

# Save as CSV inside a zip
df_base.to_csv(output_path, index=False, compression="zip")
print(f"Saved: {output_path}")

# Load later (same folder)
df = pd.read_csv(output_path, compression="zip")
grouped = df.groupby("sector")["sector"].count()
print(grouped)

Symbols pulled: 200


KeyboardInterrupt: 